In [8]:
from navec import Navec
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from TextDataset import TextDataset
from lib import vectorize_batch, train_val_test
from models.rnn_models import CreateRNN, MyGRU

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

N = 25000
EPOCHS = 5
batch_size = 128
vocab_length = 500002

In [10]:
navec = Navec.load("../models/navec_hudlit_v1_12B_500K_300d_100q.tar")

In [11]:
train_dataset, test_dataset  = TextDataset('../data/data.csv', N, 0.8, "head"), TextDataset('../data/data.csv', N, 0.2, "tail")
train_dataset, val_dataset = random_split(train_dataset, [0.8, 0.2])

train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=vectorize_batch, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, collate_fn=vectorize_batch)
val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=vectorize_batch)

In [ ]:
for X, Y in train_loader:
    print(X.shape, Y.shape)
    break

nn.Embedding(num_embeddings=vocab_length, embedding_dim=300)(X).shape

torch.randn(3, 128, 50)[1][4].shape

torch.Size([128, 50]) torch.Size([128])


In [ ]:
rnn = CreateRNN(nn.RNN).to(device)
lstm = CreateRNN(nn.LSTM).to(device)
gru = CreateRNN(nn.GRU).to(device)
mygru = CreateRNN(MyGRU).to(device)

In [ ]:
train_val_test(EPOCHS, train_loader, val_loader, test_loader, rnn, device)

In [ ]:
train_val_test(EPOCHS, train_loader, val_loader, test_loader, lstm, device)

In [ ]:
train_val_test(EPOCHS, train_loader, val_loader, test_loader, gru, device)

In [ ]:
train_val_test(EPOCHS, train_loader, val_loader, test_loader, mygru, device)